In [1]:
import pandas as pd
import domainAPIAccess as domain
import numpy as np
import json

In [2]:
df = domain.concat_raw_data(r'C:\dev\data\NSW_first_scrap')

In [3]:
print(f'Total rows: {len(df)}')
print(f'Number of duplicates: {np.sum(df.duplicated("id"))}')

df.drop_duplicates('id', inplace=True)

print(f'Number of unique rows: {len(df)}')

display(df.columns)

Total rows: 37443
Number of duplicates: 14006
Number of unique rows: 23437


Index(['listingType', 'id', 'projectId', 'media', 'headline',
       'summaryDescription', 'hasFloorplan', 'hasVideo', 'labels',
       'dateListed', 'listingSlug', 'advertiser.type', 'advertiser.id',
       'advertiser.name', 'advertiser.preferredColourHex',
       'advertiser.contacts', 'priceDetails.displayPrice',
       'propertyDetails.state', 'propertyDetails.features',
       'propertyDetails.propertyType', 'propertyDetails.allPropertyTypes',
       'propertyDetails.bathrooms', 'propertyDetails.bedrooms',
       'propertyDetails.unitNumber', 'propertyDetails.streetNumber',
       'propertyDetails.street', 'propertyDetails.area',
       'propertyDetails.region', 'propertyDetails.suburb',
       'propertyDetails.postcode', 'propertyDetails.displayableAddress',
       'propertyDetails.latitude', 'propertyDetails.longitude',
       'inspectionSchedule.byAppointment', 'inspectionSchedule.recurring',
       'inspectionSchedule.times', 'propertyDetails.carspaces',
       'advertiser.lo

In [4]:
df['converted_price'] = df['priceDetails.displayPrice'].apply(domain.convert_price_to_dollars)

df['price'] = df['priceDetails.price'].fillna(df['converted_price'])

fillnavalues = {
    'propertyDetails.bathrooms': 0,
    'propertyDetails.bedrooms': 0,
    'propertyDetails.carspaces': 0
}

df.fillna(fillnavalues, inplace=True)

df.loc[(df['price'] < 25000) | (df['price'] > 10000000), 'price'] = np.nan

display(df.describe())

display(df[['listingSlug', 'priceDetails.displayPrice', 'price']].sort_values('price').head(20))

,id,projectId,advertiser.id,propertyDetails.bathrooms,propertyDetails.bedrooms,propertyDetails.latitude,propertyDetails.longitude,propertyDetails.carspaces,priceDetails.price,priceDetails.priceFrom,priceDetails.priceTo,propertyDetails.buildingArea,propertyDetails.landArea,converted_price,price
count,2.343700e+04,662.000000,23437.000000,23437.000000,23437.000000,23433.000000,23433.000000,23437.000000,4.321000e+03,4.321000e+03,4.321000e+03,4.234000e+03,1.399800e+04,1.312100e+04,1.310800e+04
mean,2.017152e+09,4253.820242,19436.016427,1.711909,2.876904,-33.315637,150.695157,1.763835,9.675536e+05,9.675536e+05,9.695001e+05,3.977610e+03,3.740341e+05,5.628629e+07,8.745383e+05
std,7.763422e+05,1121.281082,10027.861382,2.182148,2.958219,1.777272,1.916804,2.251982,1.522616e+07,1.522616e+07,1.522627e+07,2.350138e+05,1.155203e+07,6.329285e+09,8.132295e+05
min,2.005917e+09,460.000000,0.000000,0.000000,0.000000,-37.248814,138.599500,0.000000,8.500000e+02,8.500000e+02,9.500000e+02,1.000000e+00,1.000000e+00,0.000000e+00,2.500000e+04
25%,2.017235e+09,3653.000000,12211.000000,1.000000,2.000000,-33.994392,150.553772,1.000000,3.200000e+05,3.200000e+05,3.200000e+05,8.800000e+01,5.500000e+02,4.250000e+05,4.250000e+05
50%,2.017402e+09,4750.000000,19390.000000,2.000000,3.000000,-33.810520,151.046173,1.000000,5.400000e+05,5.400000e+05,5.427150e+05,1.310000e+02,7.650000e+02,6.850000e+05,6.880000e+05
75%,2.017470e+09,5149.000000,28052.000000,2.000000,4.000000,-32.916863,151.283386,2.000000,8.790000e+05,8.790000e+05,8.800000e+05,2.130000e+02,2.004750e+03,9.999900e+05,9.990000e+05
max,2.017512e+09,5376.000000,35714.000000,280.000000,350.000000,-28.166348,167.983109,124.000000,1.000000e+09,1.000000e+09,1.000000e+09,1.528498e+07,1.217946e+09,7.250000e+11,1.000000e+07


,listingSlug,priceDetails.displayPrice,price
16095,lot-301-glen-innes-road-north-arm-cove-nsw-232...,"$25,000 - Great Price!!",25000.0
34230,80-bogan-street-bogan-gate-nsw-2876-2017255038,"$25,000",25000.0
22210,10-37-chinderah-bay-drive-chinderah-nsw-2487-2...,"$25,000",25000.0
34244,21-illewong-street-euabalong-west-nsw-2877-201...,"$25,000",25000.0
22211,9-37-chinderah-bay-drive-chinderah-nsw-2487-20...,"$25,000",25000.0
22229,2-37-chinderah-bay-drive-chinderah-nsw-2487-20...,"$25,000",25000.0
32608,lot-18-kerry-street-yerranderie-nsw-2787-20173...,"$25,000",25000.0
29047,1-bushlands-road-tocumwal-nsw-2714-2012232728,"FROM $25,000",25000.0
33670,4-clifton-place-cobar-nsw-2835-2011995470,"$25,000",25000.0
22226,7-37-chinderah-bay-drive-chinderah-nsw-2487-20...,"$25,000",25000.0


In [5]:
df['hasPrice'] = df['price'].notna()

print(f"{df['hasPrice'].sum()} out of {len(df)} ({df['hasPrice'].sum()/len(df)*100:.2f} per cent) properties have a advertised price.")

df.reset_index(drop=True).to_feather(r'C:\dev\data\NSW_advertised_scrapped_7_January.feather')

13108 out of 23437 (55.93 per cent) properties have a advertised price.


ValueError: feather does not support serializing a non-default index for the index; you can .reset_index() to make the index into column(s)